# Atuograd.function

## requires_grad

In [2]:
import torch

创建一个张量，设置 requires_grad=True 来跟踪与它相关的计算

In [3]:
x = torch.ones(2, 2, requires_grad=True) 
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


针对张量做一个操作

In [4]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


y 作为操作的结果被创建，所以它有 grad_fn，但x没有

In [5]:
print(y.grad_fn)
print(x.grad_fn)

None


针对 y 做更多的操作：

In [6]:
z = y * y * 3
out = z.mean()
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


.requires_grad_( ... ) 会改变张量的 requires_grad 标记。输入的标记默认为 False ，如果没有提供 相应的参数。

In [6]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)

# 修改requires_grad属性
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


## grad

现在后向传播，因为输出包含了一个标量，out.backward() 等同于 out.backward(torch.tensor(1.))。

In [7]:
out.backward()

>注：对out的反向梯度只能求一次，否则报错：

`RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.`

In [8]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


*雅可比向量积*

In [9]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2
    
print(y)

tensor([1318.5365, -357.8064,   16.1954], grad_fn=<MulBackward0>)


现在在这种情况下，y 不再是一个标量。torch.autograd 不能够直接计算整个雅可比，但是如果我 们只想要雅可比向量积，只需要简单的传递向量给 backward 作为参数。

In [10]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)
print(x.grad)

tensor([5.1200e+01, 5.1200e+02, 5.1200e-02])


通过将代码包裹在 with torch.no_grad()，来停止对从跟踪历史中 的 .requires_grad=True 的 张量自动求导。

In [11]:
print(x.requires_grad)
print((x**2).requires_grad)
with torch.no_grad():
    print((x**2).requires_grad)


True
True
False
